In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv('../data/wind_dataset/train.csv')
test = pd.read_csv('../data/wind_dataset/test.csv')

train

,ID,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s)
0,TRAIN_00000,7,2,저녁,13.97,287.78,9.84,76.10,992.08,15.98,12.16,3.82,7.66,1198.06,155.60,1.61
1,TRAIN_00001,8,21,오전,16.94,290.85,12.14,73.30,991.07,19.33,14.17,5.16,8.94,1183.67,177.00,1.68
2,TRAIN_00002,11,1,저녁,9.76,283.84,5.40,74.20,988.71,12.10,8.98,3.12,5.67,1213.22,146.20,0.73
3,TRAIN_00003,12,28,오전,5.27,277.30,2.71,83.50,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50,2.71
4,TRAIN_00004,9,26,오후,17.35,290.86,12.68,74.00,995.77,19.84,14.68,5.16,9.22,1187.40,19.34,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36576,TRAIN_36576,8,10,저녁,14.10,287.79,4.99,54.15,993.53,16.11,8.73,7.39,5.48,1200.85,292.90,0.77
36577,TRAIN_36577,2,20,오전,3.80,277.91,-1.01,70.70,988.03,8.02,5.67,2.35,3.58,1240.06,210.50,8.97
36578,TRAIN_36578,5,29,저녁,20.06,295.10,16.70,81.00,977.90,23.51,19.04,4.47,12.20,1153.24,19.22,2.58
36579,TRAIN_36579,9,10,저녁,25.65,299.81,15.30,52.81,988.39,32.98,17.41,15.56,11.03,1144.61,225.40,0.36


In [2]:
import random, os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(14) # Seed 고정

In [3]:
X = train.drop(['ID', '풍속 (m/s)'], axis=1)
y = train['풍속 (m/s)']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
test_x = test.drop(columns=['ID'])

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()S
le = le.fit(X['측정 시간대'])
X['측정 시간대'] = le.transform(X['측정 시간대'])
test_x['측정 시간대'] = le.transform(test_x['측정 시간대'])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(n_estimators=500)
lgbm.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003754 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2854
[LightGBM] [Info] Number of data points in the train set: 29264, number of used features: 14
[LightGBM] [Info] Start training from score 2.031712


LGBMRegressor(n_estimators=500)

In [15]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X, y)
xgb_pred = xgb.predict(test_x)
print(xgb_pred)

[2.414691  1.0429091 1.9366121 ... 0.9715214 2.070752  1.4378307]


In [10]:
pred = lgbm.predict(test_x)
print(pred)

[2.29300735 0.90399383 1.91324986 ... 0.88379785 1.80545875 0.88806381]


In [11]:
submission = pd.read_csv('../data/wind_dataset/sample_submission.csv')

In [16]:
submission['풍속 (m/s)'] = xgb_pred
submission.head()

,ID,풍속 (m/s)
0,TEST_00000,2.414691
1,TEST_00001,1.042909
2,TEST_00002,1.936612
3,TEST_00003,1.197713
4,TEST_00004,1.504009


In [17]:
submission.to_csv('submission.csv', index=False)